In [1]:
from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer
from qiskit.circuit.library import MCXGate, GroverOperator, MCMT, ZGate

### Oracle for marking a state 

In [42]:
marked_state = ""

In [43]:
def mark_states(marked_states):
    if not isinstance(marked_states, list):
        marked_states = [marked_states]

    # Calculates the number of qubits needed based on the state length
    num_qubits = len(marked_states[0])

    # Creates the cirucit
    oracle_circuit = QuantumCircuit(num_qubits, name="Oracle\n Circuit")

    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Identify where if any 0's there are in the marekd state
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Apply the apropriate gates base on the binary values order in the marked state
        if zero_inds:
            oracle_circuit.x(zero_inds)
            oracle_circuit.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
            oracle_circuit.x(zero_inds)
        else:
            oracle_circuit.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
    return oracle_circuit

In [44]:
oracle_circuit = mark_states(marked_state)

In [ ]:
oracle_circuit.draw(output="mpl")

### Grover's Reflection circuit

In [ ]:
relflection_circuit = QuantumCircuit(3, name="Relflection\n Circuit")
relflection_circuit = GroverOperator(relflection_circuit, name="Relflection\n Circuit")

relflection_circuit.decompose().draw(output="mpl", style="iqp")

### Final Cirucit

In [47]:
n = len(marked_state)

coin_register = QuantumRegister(n, name='coin')
position_register = QuantumRegister(n, name='position')
classical_register = ClassicalRegister(n,'meas')

test_circuit = QuantumCircuit(coin_register, position_register, classical_register)

In [ ]:
test_circuit.h(coin_register[:])
test_circuit.h(position_register[:])

In [49]:
def take_step(circuit):
    circuit.append(oracle_circuit, position_register[:])
    circuit.append(relflection_circuit, coin_register[:])
    test_circuit.swap(position_register[:],coin_register[:])
    return circuit


In [ ]:
steps = 2

for step in range(steps):
    test_circuit = take_step(test_circuit)

test_circuit.measure(position_register[:], classical_register[:])

In [ ]:
test_circuit.draw(output="mpl")

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
job = transpile(test_circuit, simulator)
result = simulator.run(job, backend=simulator, shots=1024).result()
counts = result.get_counts()
plot_histogram([counts])

### Real Quantum Computer

In [53]:
from qiskit_ibm_runtime import QiskitRuntimeService

In [ ]:
provider = QiskitRuntimeService(channel="ibm_quantum")
provider.backends()

In [ ]:
for backend in provider.backends():
    try:
        qubit_count = len(backend.properties().qubits)
    except:
        qubit_count = "simulated"

    print(f"{backend.name} : {backend.status().pending_jobs} & {qubit_count} qubits")

In [56]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [ ]:
provider = QiskitRuntimeService()
quantum_computer = provider.backend("ibm_sherbrooke")
pm = generate_preset_pass_manager(backend=quantum_computer, optimization_level=1)
isa_circuit = pm.run(test_circuit)
sampler = Sampler(quantum_computer)
job = sampler.run([isa_circuit])

In [ ]:
job.status()

In [ ]:
result = job.result()
plot_histogram([result[0].data.meas.get_counts()])

### Previous job result

In [58]:
ibm_api_key_filepath = ""
ibm_api_key = open(ibm_api_key_filepath,"r").read()

In [59]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel = 'ibm_quantum',
    instance = 'ibm-q/open/main',
    token = ibm_api_key
)

In [63]:
job_id = ""

In [64]:
previous_job = service.job(job_id)
previous_job_result = previous_job.result()

In [ ]:
plot_histogram([previous_job_result[0].data.meas.get_counts()])